In [1]:
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pdb

In [2]:
import numpy as np
from config import get_config
import argparse
from Learner import face_learner
from data.data_pipe import get_val_pair
from torchvision import transforms as trans
from tqdm import tqdm_notebook as tqdm


In [3]:
conf = get_config(training=False)
conf.batch_size=200 # Why bs_size can only be the number that divide 6000 well?

In [4]:
learner = face_learner(conf, inference=True)

{'detachAttentionGradient': True, 'fixed_str': 'ir_se50.pth', 'pretrainedMdl': 'ir_se50.pth', 'data_path': PosixPath('data'), 'work_path': PosixPath('work_space'), 'model_path': PosixPath('work_space/models'), 'log_path': PosixPath('work_space/log'), 'save_path': PosixPath('work_space/save'), 'exp_title': 'test', 'exp_comment': 'CosFace_ExpMS1M_detached_lre-4', 'input_size': [112, 112], 'embedding_size': 1568, 'use_mobilfacenet': False, 'modelType': 'ArcFace', 'net_depth': 50, 'drop_ratio': 0.6, 'net_mode': 'ir_se', 'device': device(type='cuda', index=0), 'test_transform': Compose(
    ToTensor()
    Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
), 'data_mode': 'emore', 'vgg_folder': PosixPath('data/faces_vgg_112x112'), 'ms1m_folder': PosixPath('data/faces_ms1m_112x112'), 'emore_folder': PosixPath('data/faces_webface_112x112'), 'batch_size': 200, 'USE_SOFTMAX': True, 'SOFTMAX_T': 1, 'facebank_path': PosixPath('data/facebank'), 'threshold': 1.5, 'threshold_xCos': 0.2338, 'face_li

In [5]:
import torch
def getUnitAttention():
    x = torch.ones(conf.batch_size//2, 1, 7, 7).cuda()
    x /= x.flatten(2).sum(dim=2).repeat(1, 1, x.size(2) * x.size(3)).view_as(x)
    return x

def getCorrAttention(corr):
    x = torch.from_numpy(corr).float().cuda()
    print(x.size())
    x = x.unsqueeze(0).unsqueeze(0)
    x = torch.cat(conf.batch_size//2 * [x])
    x /= x.flatten(2).sum(dim=2).repeat(1, 1, x.size(2) * x.size(3)).view_as(x)
    print(x.size())
    return x

unit_attention = getUnitAttention()
unit_attention.size()
# corr_attention = corrUnitAttention()
# corr_attention.size()


torch.Size([100, 1, 7, 7])

In [ ]:
# Test patched Cos
exdir = 'cosPatchWoTrained'
print(exdir)
#

In [ ]:
learner.load_state(conf, 'ir_se50.pth', model_only=True, from_save_folder=True, strict=False, model_atten=False)

In [6]:
test_datasets = ['lfw', 'vgg2_fp', 'agedb_30', 
                 'calfw', 'cfp_ff', 'cfp_fp', 'cplfw']
def testWithDatasets(test_datasets, conf, learner, attention):
    for test_dataset in test_datasets:
        pair_imgs, pair_issame = get_val_pair(conf.emore_folder, test_dataset)
        accuracy, best_threshold, roc_curve_tensor = learner.evaluate_attention(conf, 
                                                                      pair_imgs, 
                                                                      pair_issame, 
                                                                      nrof_folds=10, 
                                                                      tta=True, 
                                                                      attention=attention)
        print(test_dataset + ' - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
        trans.ToPILImage()(roc_curve_tensor)

In [ ]:
testWithDatasets(test_datasets, conf, learner, unit_attention)

In [ ]:
# corrPlot = learner.plot_CorrBtwXCosAndCos(conf, 
#                                           lfw, lfw_issame, 
#                                           nrof_folds=10, tta=True, 
#                                           attention=unit_attention)
# trans.ToPILImage()(corrPlot)

In [ ]:
# lfw, lfw_issame = get_val_pair(conf.emore_folder, 'lfw')
# learner.plot_Examples(conf, 
#                       lfw, lfw_issame, 
#                       nrof_folds=10, tta=True, 
#                       attention=unit_attention,
#                       exDir=exdir)


In [8]:
# Test patched Cos
exdir = 'cosPatchFtWithMs1M_uniAtten_cos'
print(exdir)
#

cosPatchFtWithMs1M_uniAtten_cos


In [11]:
mdl_name = '2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None.pth'
mdl_name = '2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace.pth'
learner.load_state(conf, mdl_name, model_only=True, from_save_folder=True, strict=True, model_atten=True)

In [13]:
corr_eff = np.load('2019-08-25-14-35_accuracy:0.9931666666666666_step:218349_None.npy')
corr_eff = np.load('2019-09-02-08-21_accuracy:0.9968333333333333_step:436692_CosFace.npy')
corr_attention = getCorrAttention(corr_eff)
testWithDatasets(test_datasets, conf, learner, unit_attention)
testWithDatasets(test_datasets, conf, learner, corr_attention)


testWithDatasets(test_datasets, conf, learner, None)

torch.Size([7, 7])
torch.Size([100, 1, 7, 7])
lfw - accuray:0.9958333333333333, threshold:0.23250000000000118
vgg2_fp - accuray:0.9385999999999999, threshold:0.10800000000000107
agedb_30 - accuray:0.9541666666666666, threshold:0.14450000000000102
calfw - accuray:0.9476666666666667, threshold:0.18850000000000103
cfp_ff - accuray:0.9945714285714287, threshold:0.19000000000000106
cfp_fp - accuray:0.9369999999999999, threshold:0.09550000000000107
cplfw - accuray:0.8935000000000002, threshold:0.15500000000000114
lfw - accuray:0.9960000000000001, threshold:0.2350000000000012
vgg2_fp - accuray:0.9396000000000001, threshold:0.1110000000000011
agedb_30 - accuray:0.9528333333333332, threshold:0.14850000000000102
calfw - accuray:0.9476666666666667, threshold:0.18850000000000103
cfp_ff - accuray:0.9947142857142858, threshold:0.19400000000000114
cfp_fp - accuray:0.9364285714285714, threshold:0.09750000000000103
cplfw - accuray:0.8925000000000001, threshold:0.1540000000000011
lfw - accuray:0.9960000

In [ ]:
# corrPlot = learner.plot_CorrBtwXCosAndCos(conf, 
#                                           lfw, lfw_issame, 
#                                           nrof_folds=10, tta=True, 
#                                           attention=unit_attention)
# trans.ToPILImage()(corrPlot)

In [ ]:
# lfw, lfw_issame = get_val_pair(conf.emore_folder, 'lfw')
# learner.plot_Examples(conf, 
#                       lfw, lfw_issame, 
#                       nrof_folds=10, tta=True, 
#                       attention=unit_attention,
#                       exDir=exdir)


In [ ]:
input()

In [ ]:
# Test patched Cos
exdir = 'cosPatchFtWithMs1M_learnedAtten'

#

In [ ]:
lfw, lfw_issame = get_val_pair(conf.emore_folder, 'lfw')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate_attention(conf, 
                                                                        lfw, lfw_issame, 
                                                                        nrof_folds=10, tta=True, 
                                                                        attention=None)
print('lfw - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
corrPlot = learner.plot_CorrBtwXCosAndCos(conf, 
                                          lfw, lfw_issame, 
                                          nrof_folds=10, tta=True, 
                                          attention=None)
trans.ToPILImage()(corrPlot)


In [ ]:
input()

In [ ]:
conf.use_mobilfacenet = True

In [ ]:
learner = face_learner(conf, inference=True)

In [ ]:
learner.load_state(conf, 'mobilefacenet.pth', True, True)

In [ ]:
vgg2_fp, vgg2_fp_issame = get_val_pair(conf.emore_folder, 'vgg2_fp')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, vgg2_fp, vgg2_fp_issame, nrof_folds=10, tta=True)
print('vgg2_fp - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
agedb_30, agedb_30_issame = get_val_pair(conf.emore_folder, 'agedb_30')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, agedb_30, agedb_30_issame, nrof_folds=10, tta=True)
print('agedb_30 - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
calfw, calfw_issame = get_val_pair(conf.emore_folder, 'calfw')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, calfw, calfw_issame, nrof_folds=10, tta=True)
print('calfw - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
cfp_ff, cfp_ff_issame = get_val_pair(conf.emore_folder, 'cfp_ff')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, cfp_ff, cfp_ff_issame, nrof_folds=10, tta=True)
print('cfp_ff - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
cfp_fp, cfp_fp_issame = get_val_pair(conf.emore_folder, 'cfp_fp')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, cfp_fp, cfp_fp_issame, nrof_folds=10, tta=True)
print('cfp_fp - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
cplfw, cplfw_issame = get_val_pair(conf.emore_folder, 'cplfw')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, cplfw, cplfw_issame, nrof_folds=10, tta=True)
print('cplfw - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)

In [ ]:
lfw, lfw_issame = get_val_pair(conf.emore_folder, 'lfw')
accuracy, best_threshold, roc_curve_tensor = learner.evaluate(conf, lfw, lfw_issame, nrof_folds=10, tta=True)
print('lfw - accuray:{}, threshold:{}'.format(accuracy, best_threshold))
trans.ToPILImage()(roc_curve_tensor)